# Biomass Mapping with gridFIA

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mihiarc/fiatools/blob/main/tutorials/02_biomass_mapping_with_gridfia.ipynb)
[![FIAtools](https://img.shields.io/badge/FIAtools-Ecosystem-2E7D32)](https://fiatools.org)

This tutorial shows you how to create forest biomass and species diversity maps using **gridFIA** and USDA Forest Service BIGMAP data.

## What You'll Learn

- What BIGMAP data is and its capabilities
- How to download species-level biomass data
- How to calculate diversity indices (Shannon, Simpson)
- How to create publication-ready maps
- How to export data for GIS software

## Prerequisites

- Python 3.9 or higher
- Basic familiarity with geospatial concepts

---

## 1. What is BIGMAP?

BIGMAP (Biomass Informed by Geospatial Mapping and Allometric Product) is a USDA Forest Service dataset that provides:

- **30-meter resolution** biomass maps for the contiguous US
- **327 tree species** with individual biomass layers
- **2018 base year** derived from Landsat imagery and FIA plot data
- **Tons per acre** biomass units for each pixel

### Why gridFIA?

gridFIA provides:

- **Simple Python API** for downloading and analyzing BIGMAP data
- **Zarr storage** for efficient multi-species analysis
- **Built-in diversity calculations** (Shannon, Simpson, species richness)
- **Publication-ready visualizations**

## 2. Installation

Install gridFIA from PyPI:

In [ ]:
# Install gridFIA (uncomment to run)
# !pip install gridfia

## 3. Quick Start: The GridFIA API

gridFIA uses a single entry point - the `GridFIA` class:

In [ ]:
from gridfia import GridFIA

# Initialize the API
api = GridFIA()

# List available species
species = api.list_species()
print(f"Available species: {len(species)}")

# Show first few species
for s in species[:5]:
    print(f"  {s.code}: {s.common_name}")

## 4. Downloading Species Data

Download biomass data for specific species in a geographic area:

In [ ]:
# Download loblolly pine and red maple for a North Carolina county
# files = api.download_species(
#     state="North Carolina",
#     county="Wake",
#     species_codes=["0131", "0316"],  # Loblolly pine, Red maple
#     output_dir="data/wake_county"
# )
# print(f"Downloaded {len(files)} species files")

### Common Species Codes

| Code | Species | Region |
|------|---------|--------|
| 0131 | Loblolly pine | Southeast |
| 0121 | Longleaf pine | Southeast |
| 0202 | Douglas-fir | Pacific Northwest |
| 0316 | Red maple | Eastern US |
| 0802 | White oak | Eastern US |
| 0833 | Northern red oak | Eastern US |

Use `api.list_species()` to see all 327 available species.

## 5. Creating a Zarr Store

Combine downloaded rasters into a Zarr store for efficient analysis:

In [ ]:
# Create Zarr store from downloaded files
# zarr_path = api.create_zarr(
#     input_dir="data/wake_county",
#     output_path="data/wake_county.zarr"
# )
# print(f"Created Zarr store: {zarr_path}")

### Why Zarr?

Zarr is a cloud-optimized format that:
- Stores multi-species data efficiently
- Enables lazy loading for large datasets
- Supports parallel processing
- Works with xarray for analysis

## 6. Calculating Diversity Metrics

gridFIA includes built-in calculations for forest diversity:

In [ ]:
# Calculate diversity metrics
# results = api.calculate_metrics(
#     zarr_path="data/wake_county.zarr",
#     calculations=[
#         "species_richness",    # Number of species per pixel
#         "shannon_diversity",   # Shannon-Wiener index
#         "simpson_diversity",   # Simpson's diversity index
#         "total_biomass"        # Sum of all species
#     ]
# )
# 
# # View results
# for name, data in results.items():
#     print(f"{name}: min={data.min():.2f}, max={data.max():.2f}")

### Diversity Indices Explained

| Index | Formula | Interpretation |
|-------|---------|----------------|
| **Species Richness** | Count of species | Simple count, ignores abundance |
| **Shannon-Wiener (H')** | -Σ(pᵢ × ln(pᵢ)) | Higher = more diverse, considers evenness |
| **Simpson's (D)** | 1 - Σ(pᵢ²) | 0-1 scale, probability two trees differ |

Where pᵢ = proportion of biomass from species i

## 7. Creating Maps

Generate publication-ready visualizations:

In [ ]:
# Create diversity map
# maps = api.create_maps(
#     zarr_path="data/wake_county.zarr",
#     map_type="diversity",
#     output_dir="maps/"
# )
# print(f"Created {len(maps)} map files")

In [ ]:
# Create biomass map for a specific species
# maps = api.create_maps(
#     zarr_path="data/wake_county.zarr",
#     map_type="species",
#     species_code="0131",  # Loblolly pine
#     output_dir="maps/"
# )

## 8. Exporting to GIS

Export results to GeoTIFF for use in ArcGIS, QGIS, or other GIS software:

In [ ]:
# Export diversity layer to GeoTIFF
# api.export_geotiff(
#     zarr_path="data/wake_county.zarr",
#     layer="shannon_diversity",
#     output_path="exports/shannon_diversity.tif"
# )

## 9. Complete Example: State-Level Analysis

Here's a complete workflow for analyzing a state:

In [ ]:
def analyze_county_diversity(state: str, county: str, species_codes: list):
    """Complete diversity analysis for a county."""
    from gridfia import GridFIA
    import os
    
    api = GridFIA()
    
    # Create output directories
    data_dir = f"data/{state.lower()}_{county.lower()}"
    os.makedirs(data_dir, exist_ok=True)
    
    # 1. Download species data
    print(f"Downloading {len(species_codes)} species for {county}, {state}...")
    files = api.download_species(
        state=state,
        county=county,
        species_codes=species_codes,
        output_dir=data_dir
    )
    
    # 2. Create Zarr store
    print("Creating Zarr store...")
    zarr_path = api.create_zarr(
        input_dir=data_dir,
        output_path=f"{data_dir}.zarr"
    )
    
    # 3. Calculate metrics
    print("Calculating diversity metrics...")
    results = api.calculate_metrics(
        zarr_path=zarr_path,
        calculations=["species_richness", "shannon_diversity", "total_biomass"]
    )
    
    # 4. Create maps
    print("Creating maps...")
    maps = api.create_maps(
        zarr_path=zarr_path,
        map_type="diversity",
        output_dir=f"maps/{state.lower()}_{county.lower()}"
    )
    
    # 5. Print summary
    print(f"\n{'='*50}")
    print(f"Diversity Analysis: {county}, {state}")
    print(f"{'='*50}")
    print(f"Species analyzed: {len(species_codes)}")
    print(f"Total biomass: {results['total_biomass'].sum():,.0f} tons")
    print(f"Mean Shannon diversity: {results['shannon_diversity'].mean():.2f}")
    print(f"Maps saved to: maps/{state.lower()}_{county.lower()}/")
    
    return results

# Uncomment to run:
# results = analyze_county_diversity(
#     state="North Carolina",
#     county="Wake",
#     species_codes=["0131", "0121", "0316", "0802"]
# )

## 10. Tips and Best Practices

### Memory Management
- Start with county-level analysis before scaling to states
- Use Zarr's lazy loading for large areas
- Process species in batches for state-level downloads

### Performance
- Download during off-peak hours for faster speeds
- Cache Zarr stores for repeated analysis
- Use parallel downloads when available

### Data Quality
- Check for NoData values in results
- Verify CRS matches your other data layers
- Remember BIGMAP is 2018 data - forests change!

## 11. Next Steps

Now that you can create biomass maps, explore more:

### Advanced gridFIA
- Custom bounding boxes for specific areas
- Time series analysis (when available)
- Integration with climate data

### Other FIAtools
- **[pyFIA](https://fiatools.org/tools/pyfia/)**: Query plot-level FIA data
- **[pyFVS](https://fiatools.org/tools/pyfvs/)**: Simulate forest growth
- **[askFIA](https://fiatools.org/tools/askfia/)**: Natural language queries

### Resources
- [gridFIA Documentation](https://mihiarc.github.io/gridfia/)
- [BIGMAP Data Portal](https://apps.fs.usda.gov/arcx/rest/services/RDW_Biomass)
- [FIA Species Codes](https://www.fia.fs.usda.gov/library/field-guides-methods-proc/)

---

**Questions or feedback?** Open an issue on [GitHub](https://github.com/mihiarc/gridfia/issues).